__Info__

For an enhanced experience with this notebook, particularly for viewing and creating diagrams directly within Visual Studio Code, it is highly recommended to install the "Eraser Diagrams" extension. This extension allows for the preview and creation of Eraser diagrams seamlessly within your IDE environment.

[Install Eraser Diagrams Extension](https://marketplace.visualstudio.com/items?itemName=EraserLabs.eraserlabs)

In [ ]:
#r "nuget: Microsoft.SemanticKernel"

#!import config/Settings.cs
#!import plugins/FilesPlugin.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

In [ ]:
using System.IO;

public class Architecture
{
    public string Request { get; set; }
    public string Response { get; set; }
} 
 
static string ParseFile(string filePath)
{
    var fileContent = File.ReadAllText(filePath);

        // Splitting the content into Request and Response
        var sections = fileContent.Split(new string[] { "#Request", "#Response" }, StringSplitOptions.RemoveEmptyEntries);
        if (sections.Length >= 2)
        {
            var setup = new Architecture
            {
                Request = sections[0].Trim(),
                Response = sections[1].Trim()
            };

            // Serialize to JSON
        return JsonSerializer.Serialize(setup);
    }
    else
    {
        throw new Exception("Invalid file format");
    }
}

__System Prompt Grounding__

Compared to the other samples the system prompt is a little bit more complicated.

Instead of just giving the model a persona and some chracteristics, [Grounding](https://techcommunity.microsoft.com/t5/fasttrack-for-azure/grounding-llms/ba-p/3843857) is used to teach the model how to create azure architectures using eraserdiagramm.

Beside of adding sample architectures also a list of [icons](https://docs.eraser.io/docs/icons) supported gets added to the systemprompt. This helps the model to find the right icons for the resources in the diagramm.


In [ ]:
var files = Directory.GetFiles(".\\data\\eraser\\samples");
var archs = files.Select(f => ParseFile(f)).ToList();

var icons = File.ReadAllText(".\\data\\eraser\\icons.json");

var eraserdiagram_template_promnpt = 
$"""
You are an expert in creating earaserdiagrams. 
Make sure that all open open cury braces are closed with a closing brace.
Make sure the first line always is cloud-architecture-diagram.
Also try to select the most appropriate icons for each resource based on the provided icons.

For reference you can use the following samples:

## Samples:
{JsonSerializer.Serialize(archs)}


## Icons:
{icons}
""";


In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0010

var settings = Settings.LoadFromFile("config/settings.json");

var builder = Kernel.CreateBuilder();

builder.AddAzureOpenAIChatCompletion(settings.model, settings.azureEndpoint, settings.apiKey)
    .Plugins.AddFromType<FilesPlugin>();

var kernel = builder.Build();
var chatGPT = kernel.GetRequiredService<IChatCompletionService>();

In [ ]:
var chat = new ChatHistory(eraserdiagram_template_promnpt);

In [ ]:
var promptSettings = new OpenAIPromptExecutionSettings()
    {
        MaxTokens = 4000,
        ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions,
    };

Func<string, Task> Chat = async (string input) => {
    chat.AddUserMessage(input);

     StringBuilder sb = new();
    await foreach (var message in chatGPT.GetStreamingChatMessageContentsAsync(chat, promptSettings, kernel))
    {
        sb.Append(message.Content);
    }
    var assistantReply = sb.ToString();
    chat.AddAssistantMessage(assistantReply);

    Console.WriteLine($"\nBot:");
    Console.WriteLine($"{assistantReply}\n");
};

In [ ]:
#pragma warning disable SKEXP0001

var user_prompt= """
I would like to create an Azure architecture diagram. 
The architecture should include a subscription with two ResourceGroups. 
Each ResourceGroup will contain Azure Function apps, a storage account, and a SQL Database.
All the Function apps should be connected to a single key vault, which will be deployed in a separate ResourceGroup named SharedResource. 
The SharedResource group will also include an Azure LogAnalytics workspace and Application Insights, both of which will be utilized by the Function apps.
Additionally, the Function apps will be connected to the SQL Database and the storage account within their respective ResourceGroups. 
""";


await Chat(user_prompt);


In [ ]:
await Chat("Please write the code to a file called 'azure-architecture.eraserdiagram' in a subfolder called architectures in the current directory. ");